# Building LLM Agents: A Tutorial

Welcome to this tutorial on building LLM agents. In this tutorial, we will use W&B Prompts to monitor and debug our experiments. This agent will be a great tool to help us prepare for machine learning interviews.

Our agent will be using OpenAI GPT 3.5 LLM, so we need to provide it with OpenAI API key. 

In [1]:
from getpass import getpass
import os
import openai

if os.getenv("OPENAI_API_KEY") is None:
  if any(['VSCODE' in x for x in os.environ.keys()]):
    print('Please enter password in the VS Code prompt at the top of your VS Code window!')
  os.environ["OPENAI_API_KEY"] = getpass("Paste your OpenAI key from: https://platform.openai.com/account/api-keys\n")
  openai.api_key = os.getenv("OPENAI_API_KEY", "")

assert os.getenv("OPENAI_API_KEY", "").startswith("sk-"), "This doesn't look like a valid OpenAI API key"
print("OpenAI API key configured")

Please enter password in the VS Code prompt at the top of your VS Code window!
OpenAI API key configured


## Tracking and debugging with W&B

Let's be honest - LLM agents can be very brittle! We'll use W&B to trace agent execution chains, track our experiments and debug errors. 

In [58]:
import wandb
wandb.init(project="llm-agent")
os.environ["LANGCHAIN_WANDB_TRACING"] = "true"

## Langchain

To make our life easier, we'll use Langchain agent implementation.

In [60]:
from typing import Optional

from langchain.agents import AgentType, initialize_agent
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import HumanMessage, SystemMessage
from langchain.tools import BaseTool
from langchain.tools import BaseTool
from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)

## Agent Tools

To give our agent some agency, let's define a tool that will track the user status for us. It will accept a recap of user message, retrieve previous summary from file, and update it using an LLM. This is a very naive implementation, but our goal is to learn about agents, so let's not worry about that. 

In [78]:
class UserStatusSummaryTool(BaseTool):
    name = "user_status_summary"
    description = "Input a summary of the users' message. Return an updated summary of the user's status."

    def _run(
        self, query: str, run_manager: Optional[CallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool."""
        # Read a previous user summary from a local text file
        with open('user_summary.txt', 'r') as file:
            previous_summary = file.read()
        
        # Concatenate it in a prompt with the recap
        prompt = \
f"""Please update the user summary based on the new information.\n
Previous summary: \n
{previous_summary}\n
{query}\n
Updated summary: \n"""

        # LLM to update the summary
        llm = ChatOpenAI(temperature=0.7)

        messages = [
            SystemMessage(
                content="You're a concise writer that summarizes user status in the shortest way possible, without missing relevant details."
            ),
            HumanMessage(
                content=prompt
            ),
        ]
        
        # Pass the prompt to OpenAI LLM to get an updated summary
        response = llm(messages)
        
        # Save the updated summary back to the same text file
        with open('user_summary.txt', 'w') as file:
            file.write(response.content)
        
        return response.content

    async def _arun(
        self, query: str, run_manager: Optional[AsyncCallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("pick_world does not support async")

Let's test if the tool works. 

In [79]:
recap = "user is a advanced MLE interested in NLP"
tool = UserStatusSummaryTool()
tool.run(recap)

'Advanced MLE with specific interest in NLP. No new information available.'

## Agent

Now we'll initialize a zero shot agent with OpenAI LLM and our `user_status_summary` tool. The agent should figure out what to do given a user message, and understanding user status. 

In [80]:
llm = ChatOpenAI(temperature=0.7)
tools = [UserStatusSummaryTool()]
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

## Agent Prompt Template

Remember our tutorial about Prompt Engineering? If you check out the basic agent prompt template, it's very modest, so we'll update it based on our prompt engineering experiments. 

In [81]:
print(agent.agent.llm_chain.prompt.template) 

Answer the following questions as best you can. You have access to the following tools:

user_status_summary: Input a summary of the users' message. Return an updated summary of the user's status.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [user_status_summary]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [86]:
system_prompt = """You are an AI tutor helping students prepare for machine learning coding interviews.
Your goal is understand the user needs, and to come up with learning assignments that will help students pass interviews. Specifically:
- Prompt students to solve a task that involves a simple machine learning concept and a coding exercise
- The task should be possible to solve in 30 minutes using a simple algorithm in Python
- The instruction should be minimal. Don't provide hints at this stage. 
- The task should be solvable by a student who has taken a machine learning course and has some coding experience
- The task should be interesting and fun to solve
- The task should advance the student's knowledge of machine learning

Example tasks by level:
- Beginner: calculate probability of 3 heads in 5 coin flips, count the number of times a word appears in a text
- Intermediate: implement a single neuron in Python, implement a simple decision tree in Python
- Advanced: implement backpropagation of a simple MLP in Python, implement a simple CNN in Python

You'll be evaluated on:
- your ability to summarize the user's status
- properly using the tools and passing appropriate inputs
- conciseness of the task description
- clarity of the task description
- creativity of the task
- matching the task to the student's level
- learning value of the task

You have access to the following tools:
user_status_summary: summarize the user status based on the agent's learning. Action input: recap of user message.

Use the following format:

Question: the message you get from a user
Thought: you should always think about what to do
Action: the action to take, should be one of [user_status_summary]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the task to suggest to a user
Final Answer: the final response consisting of a task or suggestion for the user

Begin!

Question: {input}
Thought:{agent_scratchpad}
"""
agent.agent.llm_chain.prompt.template = system_prompt

## Testing the agent

Now we can pass user messages to the agent and see how it uses the tools and responds. Make sure to check out the traces in W&B!

In [88]:
user_prompt = "Hi, can you give me an assignment? I'm just getting started."
agent.run(user_prompt)



> Entering new  chain...
I should ask the user for more information to understand their level and provide an appropriate assignment.
Action: user_status_summary
Action Input: Hi, can you give me an assignment? I'm just getting started.
Observation: User is a beginner and requesting an assignment.
Thought:The user is a beginner and requesting an assignment. I should provide a simple task that introduces a basic machine learning concept and can be solved in 30 minutes using a simple algorithm in Python.

Final Answer: Sure! Here's an assignment for you: calculate the probability of getting 3 heads in 5 coin flips. You can use a simple algorithm in Python to solve this. Good luck!

> Finished chain.


"Sure! Here's an assignment for you: calculate the probability of getting 3 heads in 5 coin flips. You can use a simple algorithm in Python to solve this. Good luck!"

In [90]:
user_prompt = "I'm actually interested in NLP problems"
agent.run(user_prompt)



> Entering new  chain...
I should suggest a task related to NLP for this user who is interested in NLP problems. I can start with a beginner-level task that involves counting the number of times a word appears in a text. This task will help the user practice basic string manipulation and counting techniques in Python, which are fundamental skills in NLP.
Action: user_status_summary
Action Input: None
Observation: User is a beginner in NLP and has requested an assignment. No new information is available.
Thought:The user is a beginner in NLP and is interested in NLP problems. I should suggest a task related to NLP that is suitable for beginners. 

Final Answer: Sure! Here's a task for you: Write a Python function that takes in a text string and a word, and returns the number of times the word appears in the text. This task will help you practice basic string manipulation and counting techniques in Python, which are fundamental skills in NLP. Let me know if you have any questions or ne

"Sure! Here's a task for you: Write a Python function that takes in a text string and a word, and returns the number of times the word appears in the text. This task will help you practice basic string manipulation and counting techniques in Python, which are fundamental skills in NLP. Let me know if you have any questions or need further guidance!"

Let's finish our experimentation here. 

In [91]:
wandb.finish()

## Conclusions

In this notebook, we developed a simple LLM agent using Langchain. The agent uses a tool to store user status and builds on our previous prompt engineeering experiments to recommend AI assignments to students. We've also shown how to use W&B Tracer to monitor and debug LLM Agents!